In [9]:
import pandas as pd
import numpy as np
import time 
import os.path
import datetime
import help as hp

## 读取加工后的数据
按日期分了两个文件：  
B_train_data_group_by_hour_20170102_20170131.csv  
B_train_data_group_by_hour_20170201_20170312.csv


In [10]:
# 合并按小时统计的训练数据[step1中分成了两个文件]
by_hour_data1 = pd.read_csv("B_train_data_group_by_hour_20170102_20170131.csv")
print("by_hour_data1 rows=", by_hour_data1.shape[0])
by_hour_data2 = pd.read_csv("B_train_data_group_by_hour_20170201_20170312.csv")
print("by_hour_data2 rows=", by_hour_data2.shape[0])
hour_data=pd.concat([by_hour_data1, by_hour_data2], axis=0)
print("by_hour_data rows=", hour_data.shape[0])

#两个文件合并,还有部分数据['grid_id','date','hour']是一样的,需要再次合并统计
hour_data=hour_data.groupby(['grid_id','date','hour'],as_index=False).sum()
print("by_hour_data rows=", hour_data.shape[0])

del by_hour_data1
del by_hour_data2

hour_data.head()


by_hour_data1 rows= 22985
by_hour_data2 rows= 30915
by_hour_data rows= 53900
by_hour_data rows= 53898


,grid_id,date,hour,car_number
0,51.0,20170102,8.0,9
1,51.0,20170102,9.0,14
2,51.0,20170102,10.0,7
3,51.0,20170102,11.0,8
4,51.0,20170102,12.0,7


### 数据类型转换

In [11]:
hour_data['grid_id']=hour_data['grid_id'].astype(np.int64)
hour_data['hour']=hour_data['hour'].astype(np.int64)
hour_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53898 entries, 0 to 53897
Data columns (total 4 columns):
grid_id       53898 non-null int64
date          53898 non-null int64
hour          53898 non-null int64
car_number    53898 non-null int64
dtypes: int64(4)
memory usage: 2.1 MB


### 每个grid_id生成一份数据

In [12]:
for grid_id in range(51, 101):
    grid_data=hour_data[hour_data['grid_id'] == grid_id]
    grid_data.to_csv("B_train_data_grid_{}.csv".format(grid_id), index=None)

### 导入天气数据

In [13]:
# 导入天气数据 weather_20170102_20170326.csv
weather_data=pd.read_csv("weather_20170102_20170326.csv")
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1344 entries, 0 to 1343
Data columns (total 7 columns):
date          1344 non-null int64
hour          1344 non-null int64
temp          1344 non-null int64
humidity      1344 non-null int64
wind_speed    1344 non-null float64
weather       1344 non-null int64
dayofweek     1344 non-null int64
dtypes: float64(1), int64(6)
memory usage: 73.6 KB


In [14]:
weather_data.head()

,date,hour,temp,humidity,wind_speed,weather,dayofweek
0,20170102,8,11,94,7.2,4,1
1,20170102,9,13,88,10.8,4,1
2,20170102,10,13,88,18.0,2,1
3,20170102,11,15,77,10.8,2,1
4,20170102,12,15,72,10.8,4,1


### 合并训练数据和天气数据

In [15]:
#取合并天气数据后的,按grid_id生成的数据
def get_grid_train_data(weather_data, grid_id):
    grid_data=pd.read_csv("B_train_data_grid_{}.csv".format(grid_id))
    
    #w合并天气和训练数据,用外联的方式补全某些小时段内车辆数为0的数据
    grid_train_data=pd.merge(grid_data, weather_data, on=['date', 'hour'], how = 'outer')
    grid_train_data['grid_id']=grid_id
    
    #将NaN改成0
    grid_train_data['car_number'].fillna(0, inplace=True)
    
    grid_train_data.to_csv("B_train_data_with_weath_grid_{}.csv".format(grid_id), index=None)
    
    return grid_train_data

In [16]:
grid_train_data.columns

NameError: name 'grid_train_data' is not defined

In [ ]:
grid_train_data.info()

### 生成50个grid_id的训练数据,补全0数据,合并生成一个完整的数据集

In [ ]:
train_data=get_grid_train_data(weather_data, 51)  #简单起见先加载grid=51的数据
for grid_id in range(52,101):  #合并grid=1以后的数据
    grid_train_data = get_grid_train_data(weather_data, grid_id)
    train_data = pd.concat([train_data, grid_train_data], axis=0)

train_data.to_csv("B_all_train_data_20170102_20100326.csv", index=False)

In [ ]:
byday=train_data.groupby('date').count()

byday[byday['grid_id']!=800]  #16*50 一个日期的记录数应该是800条

In [ ]:
#条数不对的数据
data_0131=train_data[train_data['date']==20170131]
data_0320=train_data[train_data['date']==20170320]
data_0326=train_data[train_data['date']==20170326]

In [ ]:
data_0326.groupby('hour').count()